In [4]:
import pyspark
import os
import shutil
root_folder = "/home/trungdc/unimelb/MAST30024/asm/mast30034_2021_s2_project_1-alexdang02-1/"
data_dir = os.path.join(root_folder, "Data/Trip")


import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import *

In [2]:
from pyspark.sql import SparkSession
import warnings

# Download yellow trip record to folder: Data/yellow

In [14]:
for file in sorted(os.listdir(os.path.join(data_dir, "yellow"))):
    warnings.filterwarnings("ignore")
    spark = SparkSession.builder.getOrCreate()
    spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
    sdf = spark.read.format("csv").option("header", "true").load(os.path.join(data_dir,"yellow",file))
    size_before = sdf.count()
    while True:
        sample = sdf.sample(withReplacement=False, fraction=0.3)
        size_after = sample.count()
        if 0.2 < size_after/size_before <= 0.3:
            print(f"File {file} is sampled with size {sample.count()/size_before} or {size_after} rows ")
            sample.repartition(1).write.csv(os.path.join(data_dir,"sample", file), header=True)
            for outfile in os.listdir(os.path.join(data_dir, "sample", file)):
                if outfile.endswith(".csv"):
                    os.rename(os.path.join(data_dir, "sample", file,outfile), os.path.join(data_dir, "after_sample", file ))
            shutil.rmtree(os.path.join(data_dir, "sample" ,file ))
            os.remove(os.path.join(data_dir, "yellow" ,file ))
            break

File yellow_tripdata_2018-01.csv is sampled with size 0.2999213230692588 or 2627273 rows 
File yellow_tripdata_2018-02.csv is sampled with size 0.2997702799645222 or 2545672 rows 
File yellow_tripdata_2018-03.csv is sampled with size 0.2999833728792998 or 2828956 rows 
File yellow_tripdata_2018-04.csv is sampled with size 0.29981102604208365 or 2789896 rows 
File yellow_tripdata_2018-05.csv is sampled with size 0.2998641704853924 or 2765966 rows 
File yellow_tripdata_2018-06.csv is sampled with size 0.29986546675050274 or 2612977 rows 
File yellow_tripdata_2018-07.csv is sampled with size 0.29974006808881 or 2352884 rows 
File yellow_tripdata_2018-08.csv is sampled with size 0.2999840491957457 or 2354615 rows 
File yellow_tripdata_2018-09.csv is sampled with size 0.2998742682490451 or 2411029 rows 
File yellow_tripdata_2018-10.csv is sampled with size 0.2999733026644621 or 2646096 rows 
File yellow_tripdata_2018-11.csv is sampled with size 0.2997749339362596 or 2441716 rows 
File yello

In [5]:
for file in sorted(os.listdir(os.path.join(data_dir, "yellow"))):
    warnings.filterwarnings("ignore")
    spark = SparkSession.builder.getOrCreate()
    spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
    sdf = spark.read.format("csv").option("header", "true").load(os.path.join(data_dir,"yellow",file))
    size_before = sdf.count()
    while True:
        sample = sdf.sample(withReplacement=False, fraction=0.3)
        size_after = sample.count()
        if 0.2 < size_after/size_before <= 0.3:
            print(f"File {file} is sampled with size {sample.count()/size_before} or {size_after} rows ")
            sample.repartition(1).write.csv(os.path.join(data_dir,"sample", file), header=True)
            for outfile in os.listdir(os.path.join(data_dir, "sample", file)):
                if outfile.endswith(".csv"):
                    os.rename(os.path.join(data_dir, "sample", file,outfile), os.path.join(data_dir, "yellow_after", file ))
            shutil.rmtree(os.path.join(data_dir, "sample" ,file ))
            os.remove(os.path.join(data_dir, "yellow" ,file ))
            break

File yellow_tripdata_2019-12.csv is sampled with size 0.2998718011367517 or 2068011 rows 
File yellow_tripdata_2020-01.csv is sampled with size 0.2998892775852134 or 5940235 rows 
File yellow_tripdata_2020-02.csv is sampled with size 0.2999190778335432 or 5803642 rows 
File yellow_tripdata_2020-03.csv is sampled with size 0.2999526637155667 or 6594534 rows 
File yellow_tripdata_2020-04.csv is sampled with size 0.2999571436247258 or 6313911 rows 
File yellow_tripdata_2020-05.csv is sampled with size 0.2998834908705921 or 6467195 rows 
File yellow_tripdata_2020-06.csv is sampled with size 0.2998749531766383 or 6337942 rows 
File yellow_tripdata_2020-07.csv is sampled with size 0.2998846615620915 or 6476442 rows 
File yellow_tripdata_2020-08.csv is sampled with size 0.2999285462081042 or 6633337 rows 
File yellow_tripdata_2020-09.csv is sampled with size 0.29982457099632503 or 6640341 rows 
File yellow_tripdata_2020-10.csv is sampled with size 0.2998564207740911 or 6982695 rows 
File yell

# Concaternate sampled data

In [6]:
warnings.filterwarnings("ignore")
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
sdf = spark.read.format("csv").option("header", "true").load(os.path.join(data_dir,"yellow_after","*.csv"))
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
sdf.createOrReplaceTempView("trip")
sdf.limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,2019-12-01 00:26:58,2019-12-01 00:41:45,1,4.20,1,N,142,116,2,14.5,3,0.5,0,0,0.3,18.3,2.5
1,2019-12-01 00:25:53,2019-12-01 00:26:04,1,.00,1,N,145,145,2,2.5,0.5,0.5,0,0,0.3,3.8,0
1,2019-12-01 00:19:48,2019-12-01 00:24:18,1,.90,1,N,148,4,1,5.5,3,0.5,1.85,0,0.3,11.15,2.5
2,2019-12-01 00:43:02,2019-12-01 01:11:18,1,13.07,1,N,41,51,2,38.5,0.5,0.5,0,0,0.3,39.8,0
2,2019-12-01 00:37:17,2019-12-01 01:07:39,5,19.98,2,N,132,238,1,52,0,0.5,14.73,6.12,0.3,73.65,0


In [7]:
print(f"Size of data after being sample: {sdf.count(), len(sdf.columns)}")

Size of data after being sample: (80271519, 18)


In [8]:
sql_query = """
SELECT *
FROM trip
ORDER BY "tpep_pickup_datetime"
"""
file = "yellow2020.csv"
out = spark.sql(sql_query)
out.head()
out.repartition(1).write.csv(os.path.join(data_dir, file), header=True)

# Note
- Go to this directory: root_folder/Trip/Data/yellow_concat.csv/
- Inside there will be a single csv file with strange name
- Move the file to this directory root_folder/Trip/Data/
- Delete the folder: root_folder/Trip/Data/yellow_concat.csv/
- Rename that strange-name csv file to yellow_concat.csv

Sorry for this inconvenience. I try to automate but run into a bug that I can not fix